In [1]:
from datetime import date
import pandas as pd
import nselib
from nselib import capital_market
from sqlalchemy import create_engine
import numpy as np
import os


In [2]:
import duckdb

motherduck_token = os.getenv("MOTHERDUCK_TOKEN")
con = duckdb.connect(f"md:?motherduck_token={motherduck_token}")

In [3]:
con.sql("SELECT date1,count(*) from daily_nse_price group by date1 order by date1 desc").df()

,date1,count_star()
0,2025-09-23,2982
1,2025-09-22,3039
2,2025-09-19,2979
3,2025-09-18,2984
4,2025-09-17,3007
...,...,...
1162,2021-01-07,2017
1163,2021-01-06,2040
1164,2021-01-05,2049
1165,2021-01-04,2055


In [4]:
allstocks= pd.read_excel("data/allstocks.xlsx")

In [5]:
NSE_ISIN = pd.read_csv("data/NSE_ISIN.csv")
NSE_ISIN.columns = NSE_ISIN.columns.str.strip()

In [6]:
df = con.sql("SELECT symbol as 'Ticker', date1 as 'Date', open_price as 'Open', high_price as 'High',low_price as 'Low',close_price as 'Close', deliv_qty as 'Volume' FROM daily_nse_price where series = 'EQ' and date1 > '2025-01-01'").df()

df["Volume"] = df["Volume"].astype(float)

In [7]:
def weekly_supertrend_daily(df, atr_period=10, multiplier=3):
    """
    df: DataFrame with ['Ticker','Date','Open','High','Low','Close']
    Returns df with 'SuperTrend' and 'Trend' columns applied to daily rows, based on weekly ATR
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Ticker','Date'])

    result_list = []

    for ticker, group in df.groupby('Ticker'):
        # Resample weekly to get High, Low, Close for SuperTrend calculation
        weekly = group.resample('W-FRI', on='Date').agg({
            'High':'max',
            'Low':'min',
            'Close':'last'
        }).sort_index()

        # Calculate weekly ATR
        weekly['H-L'] = weekly['High'] - weekly['Low']
        weekly['H-Cp'] = abs(weekly['High'] - weekly['Close'].shift())
        weekly['L-Cp'] = abs(weekly['Low'] - weekly['Close'].shift())
        weekly['TR'] = weekly[['H-L','H-Cp','L-Cp']].max(axis=1)
        weekly['ATR'] = weekly['TR'].rolling(atr_period, min_periods=1).mean()

        # Basic bands
        weekly['Basic_Up'] = (weekly['High'] + weekly['Low'])/2 + multiplier*weekly['ATR']
        weekly['Basic_Down'] = (weekly['High'] + weekly['Low'])/2 - multiplier*weekly['ATR']

        # Weekly SuperTrend
        weekly['SuperTrend'] = 0.0
        for i in range(1,len(weekly)):
            prev = weekly.iloc[i-1]
            if prev['SuperTrend'] < prev['Close']:
                curr_st = max(weekly.iloc[i]['Basic_Down'], prev['SuperTrend'])
            else:
                curr_st = min(weekly.iloc[i]['Basic_Up'], prev['SuperTrend'])
            weekly.at[weekly.index[i], 'SuperTrend'] = curr_st

        # ---- 2. Map weekly SuperTrend to daily data ----
        group = group.set_index('Date')
        group['SuperTrend'] = weekly['SuperTrend'].reindex(group.index, method='ffill')

        # ---- 3. Daily Trend based on daily Close vs weekly SuperTrend ----
        group['Trend'] = group['Close'] > group['SuperTrend']

        group['Ticker'] = ticker
        result_list.append(group.reset_index())

    return pd.concat(result_list, ignore_index=True)

In [8]:
daily_with_weekly_st = weekly_supertrend_daily(df, atr_period=10, multiplier=2)


In [9]:
latest_supertrend = daily_with_weekly_st.groupby("Ticker").tail(1)

In [10]:
print(latest_supertrend)

             Date      Ticker      Open      High       Low     Close  \
170    2025-09-23   20MICRONS    228.50    232.00    225.25    226.24   
289    2025-09-23  21STCENMGM     53.39     53.40     51.55     52.87   
467    2025-09-23      360ONE   1048.00   1053.30   1015.40   1019.60   
645    2025-09-23   3IINFOLTD     24.30     24.60     24.30     24.38   
823    2025-09-23     3MINDIA  30000.00  30085.00  29780.00  29835.00   
...           ...         ...       ...       ...       ...       ...   
367974 2025-09-23        ZOTA   1438.00   1475.00   1412.20   1466.20   
368152 2025-09-23       ZUARI    270.95    281.00    268.10    274.20   
368330 2025-09-23    ZUARIIND    287.95    331.95    287.95    313.40   
368508 2025-09-23   ZYDUSLIFE   1035.50   1048.00   1030.30   1044.85   
368686 2025-09-23   ZYDUSWELL    496.30    501.30    471.20    473.80   

           Volume    SuperTrend  Trend  
170       41932.0    221.788000   True  
289        3526.0     59.876000  False  


In [11]:
price_threshold = 0.03
volume_threshold = 2.0

latest_date = df["Date"].max()

In [12]:
results = []

for ticker, group in df.groupby("Ticker"):
    group = group.sort_values("Date").reset_index(drop=True)

    # Calculate daily % price change (Close vs Previous Close)
    group["Pct_Change"] = group["Close"].pct_change()

    # Calculate rolling average volume (last 20 days)
    group["Avg_Volume"] = group["Volume"].rolling(window=20, min_periods=5).mean()

    # Signal: price up > threshold & volume > threshold * avg_volume
    group["Signal"] = (group["Pct_Change"] > price_threshold) & (group["Volume"] > volume_threshold * group["Avg_Volume"])

    results.append(group)

result_df = pd.concat(results)

signals = result_df[result_df["Signal"]]
latest_signals = signals[signals["Date"] == latest_date]

In [13]:
latest_signals = latest_signals.merge(
    NSE_ISIN[["SYMBOL","ISIN NUMBER"]],
    left_on="Ticker",
    right_on="SYMBOL",
    how="inner"
)

In [14]:
latest_signals = pd.merge(
    latest_signals,
    allstocks,
    left_on="ISIN NUMBER",         # column name in latest_signals
    right_on="ISIN",     # column name in allstocks
    how="left"           # inner join (only matching tickers)
)

In [15]:
latest_signals = latest_signals.merge(
    latest_supertrend[["Ticker","SuperTrend","Trend"]],
    on="Ticker",         # column name in latest_signals
    how="left"           # inner join (only matching tickers)
)

In [33]:
filtered.shape

(0, 58)

In [16]:
filtered = latest_signals[
    (latest_signals["Market Capitalization"] > 1000) &
    (latest_signals["ROCE Annual 3Yr Avg %"] > 15) &
    (latest_signals['Long Term Debt To Equity Annual'] < 0.5) &
    (latest_signals['Promoter holding latest %'] > 60) &
    (latest_signals['Promoter holding pledge percentage % Qtr'] < 0.01) &
    (latest_signals['Net Profit Qtr Growth YoY %'] > 0) &
    (latest_signals['Operating Revenue growth TTM %'] > 15) &
    (latest_signals['Cash EPS 5Yr Growth %'] > 15) &
    (latest_signals['EPS TTM Growth %'] > 15) &
    (latest_signals["PEG TTM PE to Growth"] > 0) &
    (latest_signals["PEG TTM PE to Growth"] < 2)
]

In [31]:
filtered.to_csv("my_data.csv", index=False)

In [25]:
print(allstocks[allstocks['NSE Code']=='GUJTHEM'])

Empty DataFrame
Columns: [Stock Name, NSE Code, BSE Code, Stock Code, ISIN, Industry Name, sector_name, Current Price, Current Price.1, Market Capitalization, Trendlyne Durability Score, Trendlyne Valuation Score, Trendlyne Momentum Score, DVM_classification_text, Prev Day Trendlyne Durability Score, Prev Day Trendlyne Valuation Score, Prev Day Trendlyne Momentum Score, Prev Week Trendlyne Durability Score, Prev Week Trendlyne Valuation Score, Prev Week Trendlyne Momentum Score, Prev Month Trendlyne Durability Score, Prev Month Trendlyne Valuation Score, Prev Month Trendlyne Momentum Score, Normalized Momentum Score, Market Capitalization.1, PE TTM Price to Earnings, PEG TTM PE to Growth, Price to Book Value, Operating Revenue TTM, Revenue Growth Qtr YoY %, Operating Revenue growth TTM %, EPS TTM Growth %, Cash EPS 5Yr Growth %, Net Profit Qtr Growth YoY %, ROCE Annual 3Yr Avg %, Operating Profit Margin Qtr %, Operating Profit Margin TTM %, Operating Profit Margin Annual %, Operating P

In [ ]:
allstocks.shape

(5693, 44)

In [21]:
BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
CHAT_ID = '5798902540'

def send_dataframe_via_telegram(df, bot_token, chat_id, caption="DataFrame"):
    import matplotlib.pyplot as plt
    import requests
    
    if df.empty:
        telegram = requests.post(
            f"https://api.telegram.org/bot{bot_token}/sendMessage",
            data={"chat_id": chat_id, "text": f"Nothing in {caption} list today."}
        )
        return telegram.json()
    # Render DataFrame as image
    fig, ax = plt.subplots(figsize=(6, 0.5 + 0.3*len(df)))  # auto height
    ax.axis("off")
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc="center",
        loc="center"
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    
    filename = "table.png"
    plt.savefig(filename, bbox_inches="tight")
    plt.close()
    
    # Send to Telegram
    url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
    with open(filename, "rb") as f:
        res = requests.post(url, data={"chat_id": chat_id, "caption": caption}, files={"photo": f})
    return res.json()

In [22]:

send_dataframe_via_telegram(filtered[["Stock Name"]], BOT_TOKEN, CHAT_ID, "Buy")

{'ok': True,
 'result': {'message_id': 42,
  'from': {'id': 7252728340,
   'is_bot': True,
   'first_name': 'DailyNseDataLoad',
   'username': 'DpsStockAnalysis_bot'},
  'chat': {'id': 5798902540,
   'first_name': 'Daksh',
   'last_name': 'Singh',
   'type': 'private'},
  'date': 1758689264,
  'text': 'Nothing in Buy list today.'}}